In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when, lit, count, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidatorModel
import numpy as np
from pyspark.ml.classification import LinearSVC
from sklearn import neighbors
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.ml.linalg import Vectors
from functools import reduce
import scipy.io
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import SparseVector, DenseVector, VectorUDT

In [2]:
from pyspark import since
from pyspark import keyword_only
from pyspark.ml.param import Param
from pyspark.ml.param import Params
from pyspark.ml.param import TypeConverters
from pyspark.ml import Estimator, Transformer
from pyspark.ml.param.shared import HasCollectSubModels, HasParallelism
from pyspark.ml.tuning import ValidatorParams
from pyspark.ml.util import MLReadable, MLWritable
from pyspark.sql.functions import rand
from multiprocessing.pool import ThreadPool
from pyspark.ml.tuning import _parallelFitTasks
import numpy as np

class UnderOverCrossValidator(Estimator, ValidatorParams, HasParallelism, HasCollectSubModels, MLReadable, MLWritable):
    
    numFolds = Param(Params._dummy(), "numFolds", "number of folds for cross validation",
                     typeConverter=TypeConverters.toInt)
    
    underTransformer = Param(Params._dummy(), "underTransformer", "transformer for undersampling")
    overTransformer = Param(Params._dummy(), "overTransformer", "transformer for oversampling")

    @keyword_only
    def __init__(self, estimator=None, estimatorParamMaps=None, evaluator=None, numFolds=3,
                 seed=None, parallelism=1, collectSubModels=False, underTransformer=None,
                 overTransformer=None):
        """
        __init__(self, estimator=None, estimatorParamMaps=None, evaluator=None, numFolds=3,\
                 seed=None, parallelism=1, collectSubModels=False, underTransformer=None)
        """
        super(UnderOverCrossValidator, self).__init__()
        self._setDefault(numFolds=3, parallelism=1, underTransformer=None, overTransformer=None)
        kwargs = self._input_kwargs
        self._set(**kwargs)
    
    @staticmethod
    def __transform_data(pipeline, df, features_label='features', target_label='label'):
        data = df
        est = pipeline.copy()
        if not isinstance(est, Pipeline):
            return est, data
        first_estimator_index = -1
        stages = est.getStages()
        for i in range(len(stages)):
            if isinstance(stages[i], Estimator):
                first_estimator_index = i
                break
            data = stages[i].transform(data)
        return est.setStages(stages[first_estimator_index:]), data.select(features_label, target_label)
    
    def __transform_df(self, dataset):
        underTransformer = self.getOrDefault(self.underTransformer)
        overTransformer = self.getOrDefault(self.overTransformer)
        
        if underTransformer is not None:
            dataset.unpersist()
            dataset = underTransformer.transform(dataset).cache()
            print(f'train quantity after undersample: {dataset.count()}')
            dataset.groupby('label').agg(count(col('label'))).show()
        if overTransformer is not None:
            dataset.unpersist()
            dataset = overTransformer.transform(dataset).cache()
            print(f'train quantity after oversample: {dataset.count()}')
            dataset.groupby('label').agg(count(col('label'))).show()
        return dataset

        
    
    def _fit(self, dataset):
        est = self.getOrDefault(self.estimator)
        est_test, df = self.__transform_data(est, dataset)
        epm = self.getOrDefault(self.estimatorParamMaps)
        numModels = len(epm)
        eva = self.getOrDefault(self.evaluator)
        nFolds = self.getOrDefault(self.numFolds)
        seed = self.getOrDefault(self.seed)
        h = 1.0 / nFolds
        randCol = self.uid + "_rand"
        #df = dataset.select("*", rand(seed).alias(randCol))
        df = df.select("*", rand(seed).alias(randCol))
        metrics = [0.0] * numModels

        pool = ThreadPool(processes=min(self.getParallelism(), numModels))
        subModels = None
        collectSubModelsParam = self.getCollectSubModels()
        if collectSubModelsParam:
            subModels = [[None for j in range(numModels)] for i in range(nFolds)]

        for i in range(nFolds):
            validateLB = i * h
            validateUB = (i + 1) * h
            condition = (df[randCol] >= validateLB) & (df[randCol] < validateUB)
            validation = df.filter(condition).cache()
            train = df.filter(~condition).cache()
            #print(train.columns)
            print(f'train original quantity: {train.count()}')
            train.groupby('label').agg(count(col('label'))).show()
            tasks = _parallelFitTasks(est_test, train, eva, validation, epm, collectSubModelsParam)
            train = self.__transform_df(train)
            
            for j, metric, subModel in pool.imap_unordered(lambda f: f(), tasks):
                metrics[j] += (metric / nFolds)
                print(f'Fold {i}, iteration {j} => {metrics[j]}')
                if collectSubModelsParam:
                    subModels[i][j] = subModel

            validation.unpersist()
            train.unpersist()

        if eva.isLargerBetter():
            bestIndex = np.argmax(metrics)
        else:
            bestIndex = np.argmin(metrics)
        bestModel = est_test.fit(self.__transform_df(df), epm[bestIndex])
        
        print('Best Model: ', epm[bestIndex], 'Avg Score {}'.format(metrics[bestIndex]))
        
        return self._copyValues(CrossValidatorModel(bestModel, metrics, subModels))

In [3]:
from pyspark.ml import Transformer
class RandomUndersampler(Transformer):
    """
    A custom Transformer that applies the Random UnderSample method (RUS)
    """

    def __init__(self, ratio=1.0, minority_class=1.0, label_column='label'):
        super(RandomUndersampler, self).__init__()
        self.label_column = label_column
        self.minority_class = minority_class
        self.ratio = ratio
    
    def __rus(self, df):
        minority = df.filter(col(self.label_column) == self.minority_class)
        mayority = df.filter(col(self.label_column) != self.minority_class)
        total_minority = minority.count()
        total_mayority = mayority.count()
        fraction= np.double(total_minority * self.ratio) / np.double(total_mayority)
        sample = mayority.sample(False,fraction)

        return sample.union(minority)

    def _transform(self, df):
        df = self.__rus(df)
        return df

In [4]:
def KNN(df, k=5, algorithm='auto', features_label='features'):
    w = Window().orderBy(lit(1))
    features_vector = df.select(features_label).cache()
    knnObj = neighbors.NearestNeighbors(n_neighbors=k+1, algorithm=algorithm).fit(features_vector.rdd.map(lambda x: x[0]).collect())
    bc_knnObj = sc.broadcast(knnObj)
    results = features_vector.rdd.map(lambda x: bc_knnObj.value.kneighbors(x)).map(lambda x: (x[0][0][1:].tolist(), x[1][0][1:].tolist())).toDF()
    features_vector.unpersist()
    #bc_knnObj.destroy()
    bc_knnObj.unpersist()
    return df.withColumn('columnindex', row_number().over(w)).join(
        results.withColumnRenamed('_1', 'gap').withColumnRenamed('_2', 'neighbours').withColumn('columnindex', row_number().over(w)),
        ['columnindex'],
        'inner'
    ).drop('columnindex')

In [5]:
from pyspark.ml import Transformer
class SmoteOversampler(Transformer):
    """
    A custom Transformer that applies the Random UnderSample method (RUS)
    """

    def __init__(self, ratio=1.0, minority_class=1.0, k=5, algorithm='auto', features_label='features', target_label='label'):
        super(SmoteOversampler, self).__init__()
        self.minority_class = minority_class
        self.k = k
        self.ratio = ratio
        self.algorithm = algorithm
        self.features_label = features_label
        self.target_label = target_label
    
    def __smote(self, df):
        minority_class = self.minority_class
        k = self.k
        ratio = self.ratio
        algorithm = self.algorithm
        features_label = self.features_label
        target_label = self.target_label
        
        minority = df.where(col(target_label) == minority_class)
        mayority = df.where(col(target_label) != minority_class)

        knn = KNN(minority, k=k, algorithm=algorithm).cache()
        original_features = knn.select('features').rdd.map(lambda x: x[0]).collect()
        bc_original_features = sc.broadcast(original_features)

        mayority_count = mayority.count()
        minority_count = minority.count()
        quantity_to_generate = np.min((mayority_count * ratio) - minority_count, 0)
        generation_cycles = int(np.ceil(quantity_to_generate/minority_count))
        if generation_cycles == 0:
            return df

        generated = []
        for cycle in range(generation_cycles):
            generated.append(knn.rdd.map(
                lambda x: (
                    Vectors.dense((x['features'] + (bc_original_features.value[np.random.choice(x['neighbours'])] - x['features']) * np.random.random())),
                    x['label'])
            ).toDF().withColumnRenamed('_1', 'features').withColumnRenamed('_2', 'label'))
        generated = reduce(lambda g1, g2: g1.select(features_label, target_label).union(g2.select(features_label, target_label)), generated)
        generated = generated.sample(False, np.double(quantity_to_generate)/(minority_count * generation_cycles))
        bc_original_features.unpersist()
        knn.unpersist()
        return df.select(features_label, target_label).union(generated.select(features_label, target_label))

    def _transform(self, df):
        df = self.__smote(df)
        return df

In [6]:
def general_stats(predictionAndLabels, labels):
    metrics = MulticlassMetrics(predictionAndLabels)
    precision = metrics.precision()
    recall = metrics.recall()
    f1Score = metrics.fMeasure()
    print("Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)
    for label in sorted(labels):
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))
                 # Weighted stats
    print("Weighted recall = %s" % metrics.weightedRecall)
    print("Weighted precision = %s" % metrics.weightedPrecision)
    print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
    print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
    print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)
    
    binMetrics = BinaryClassificationMetrics(predictionAndLabels)
    # Area under precision-recall curve
    print("Area under PR = %s" % binMetrics.areaUnderPR)
    # Area under ROC curve
    print("Area under ROC = %s" % binMetrics.areaUnderROC)

In [7]:
def split_dataset(df, ratios, label_column='label'):
    dataset = df.cache()
    mayority_split = dataset.filter(dataset[label_column] == 0).randomSplit(ratios)
    minority_split = dataset.filter(dataset[label_column] > 0).randomSplit(ratios)
    dataset.unpersist()
    return (
        mayority_split[0].union(minority_split[0]),
        mayority_split[1].union(minority_split[1])
    )

In [8]:
from pyspark.ml.pipeline import Transformer  
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark.ml.linalg import SparseVector, DenseVector, VectorUDT
from pyspark.sql.functions import udf


class AsDenseTransformer(Transformer, HasInputCol, HasOutputCol):  
    def __init__(self, inputCol=None, outputCol=None):
        super(AsDenseTransformer, self).__init__()
        self.inputCol = inputCol
        self.outputCol = outputCol

    def _transform(self, dataset):
        out_col = self.outputCol
        in_col = dataset[self.inputCol]

        asDense = udf(lambda s: DenseVector(s.toArray()), VectorUDT()) 

        return dataset.withColumn(out_col,  asDense(in_col))

In [9]:
spark = SparkSession.builder.appName('spark test').getOrCreate()

In [10]:
# Para LDD
mat = scipy.io.loadmat('databases/ldd/ldd1.mat')
nmat = []
for i in range(len(mat['Y'])):
    nmat.append((*mat['X'][i].tolist(), int(mat['Y'][i][0])))
data = spark.createDataFrame(nmat)
data = data.withColumnRenamed('_11', 'label').withColumn('label', when(col('label') == -1, 0.0).otherwise(1.0))

In [11]:
# Para Agrawal
df = spark.read.csv('databases/agrawal/Agrawal1.csv', header=True, inferSchema=True)

agrawal_pipeline = Pipeline(stages=[
    StringIndexer(inputCol="class", outputCol="label"),
    StringIndexer(inputCol="zipcode", outputCol="zipcode_index"),
    StringIndexer(inputCol="elevel", outputCol="elevel_index"),
    StringIndexer(inputCol="car", outputCol="car_index"),
    OneHotEncoderEstimator(
        inputCols=['zipcode_index', 'elevel_index', 'car_index'],
        outputCols=["zipcode_vec", 'elevel_vec', 'car_vec']
    ),
    QuantileDiscretizer(numBuckets=3, inputCol="age", outputCol="q_age"),
    QuantileDiscretizer(numBuckets=3, inputCol="hyears", outputCol="q_hyears"),
    VectorAssembler(inputCols=['salary', 'commission', 'hvalue', 'loan'], outputCol='scalable_vec'),
    MinMaxScaler(inputCol="scalable_vec", outputCol="scaledFeatures")
])

data = agrawal_pipeline.fit(df).transform(df).drop(
    'class',
    'zipcode',
    'elevel',
    'zipcode_index',
    'elevel_index',
    'car_index',
    'age',
    'hyears',
    'car',
    'salary',
    'commission',
    'hvalue',
    'loan',
    'scalable_vec'
)

In [12]:
data.groupby('label').agg(count(col('label'))).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|      672045|
|  1.0|      327955|
+-----+------------+



In [13]:
train, test = split_dataset(data, [0.7, 0.3])

In [14]:
train.groupby('label').agg(count(col('label'))).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|      470306|
|  1.0|      229463|
+-----+------------+



In [15]:
test.groupby('label').agg(count(col('label'))).show()

+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|      201739|
|  1.0|       98492|
+-----+------------+



In [16]:
assembler = VectorAssembler(
    inputCols=data.drop('label').columns,
    outputCol='features'
)

In [17]:
denser = AsDenseTransformer(inputCol='features', outputCol='features')

In [18]:
rus = RandomUndersampler(minority_class=1.0, ratio=1.0)

In [19]:
smote = SmoteOversampler(minority_class=1.0, ratio=4.0)

In [20]:
smote.transform(rus.transform(denser.transform(assembler.transform(train)))).groupby('label').count().show()

+-----+------+
|label| count|
+-----+------+
|  0.0|228936|
|  1.0|915793|
+-----+------+



In [21]:
lsvc = LinearSVC()

In [22]:
lr = LogisticRegression()

In [23]:
pipeline = Pipeline(stages=[assembler, denser, lr])

In [24]:
paramGrid = ParamGridBuilder(
).addGrid(
    lr.regParam, [0.01]
).addGrid(
    lr.maxIter, [10, 100]
).build()

In [25]:
crossval = UnderOverCrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=BinaryClassificationEvaluator(),
    numFolds=2,
    underTransformer=rus,
    overTransformer=smote
)

In [26]:
cvmodel = crossval.fit(train)

train original quantity: 349665
+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|      235130|
|  1.0|      114535|
+-----+------------+

train quantity after undersample: 228661
+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|      114126|
|  1.0|      114535|
+-----+------------+

train quantity after oversample: 570631
+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|      114126|
|  1.0|      456505|
+-----+------------+

Fold 0, iteration 0 => 0.25431056138980956
Fold 0, iteration 1 => 0.2543515399271086
train original quantity: 350104
+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|      235176|
|  1.0|      114928|
+-----+------------+

train quantity after undersample: 229585
+-----+------------+
|label|count(label)|
+-----+------------+
|  0.0|      114657|
|  1.0|      114928|
+-----+------------+

train quantity after oversample: 573295
+-----+------------+
|label|count(label)|
+-----+-----

In [27]:
prediction = cvmodel.transform(assembler.transform(test))

In [28]:
prediction.groupby('label', 'prediction').agg(count('*')).show()

+-----+----------+--------+
|label|prediction|count(1)|
+-----+----------+--------+
|  1.0|       1.0|   98492|
|  0.0|       1.0|  201739|
+-----+----------+--------+



In [29]:
labels = list(map(lambda i: float(i[0]), train.select('label').distinct().collect()))

In [30]:
general_stats(prediction.rdd.map(lambda l: (float(l.prediction), float(l.label))), labels)

Summary Stats
Precision = 0.3280540650365885
Recall = 0.3280540650365885
F1 Score = 0.3280540650365885
Class 0.0 precision = 0.0
Class 0.0 recall = 0.0
Class 0.0 F1 Measure = 0.0
Class 1.0 precision = 0.3280540650365885
Class 1.0 recall = 1.0
Class 1.0 F1 Measure = 0.49403721380507276
Weighted recall = 0.3280540650365885
Weighted precision = 0.10761946958703024
Weighted F(1) Score = 0.16207091626810433
Weighted F(0.5) Score = 0.12432777868513115
Weighted false positive rate = 0.3280540650365885
Area under PR = 0.3280540650365885
Area under ROC = 0.5


In [31]:
metrics = MulticlassMetrics(prediction.rdd.map(lambda l: (float(l.prediction), float(l.label))))

In [32]:
metrics.confusionMatrix().toArray()

array([[     0., 201739.],
       [     0.,  98492.]])